In [104]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.decomposition import PCA
from sklearn.utils import shuffle

In [105]:
# Load data with specific id number
def load_data(id_num):
    data = np.load("eeg_data.npz")
    X = data['x']
    y = data['y']
    
    index = [i for i in range(len(y)) if y[i] == id_num]
    
    output_data = []
    output_label = []
    
    for i in index:
        output_data.append(X[i])
        output_label.append(y[i])
        
    return output_data, output_label

In [106]:
# Compare seeing one number with rest 
def binary_all_channel(data, label, id_num):
    if len(data) != len(label):
        print("Something is wrong here")
        return
    
    output_data = []
    output_label = []
    
    for i in range(len(label)):
        if label[i] != id_num and label[i] != -1:
            print("Something is wrong here")
            break
        if label[i] != -1:
            output_label.append([1])
        else:
            output_label.append([-1])
        
        feature = np.concatenate(data[i])
        feature = np.nan_to_num(feature)
        output_data.append(feature)
        
    return output_data, output_label    

In [107]:
def train(X, y):
    w = np.linalg.inv((X.transpose() @ X)) @ (X.transpose() @ y)
    return w

In [108]:
def binary_predict(X, w):
    raw_val = X.transpose() @ w
    
    if raw_val >= 0:
        return 1
    if raw_val < 0:
        return -1

In [109]:
def predict(X, w, mode):
    raw_val = X.transpose() @ w
    
    if mode == "binary":
        if raw_val >= 0:
            return 1
        if raw_val < 0:
            return -1

In [113]:
def cross_val(X, y, batch_size):
    error_arr = []
    subset_num = int(len(X)/batch_size)-1
    for i in range(subset_num):
        error = 0
        X_test = X[i*batch_size: (i+1)*batch_size]
        y_test = y[i*batch_size: (i+1)*batch_size]
        X_train = np.concatenate((X[0: i*batch_size], X[(i+1)*batch_size: len(X)]))
        y_train = np.concatenate((y[0: i*batch_size], y[(i+1)*batch_size: len(y)]))

        w = train(X_train, y_train)
        for i in range(len(X_test)):
            result = predict(X_test[i], w, 'binary')
            if result != y_test[i]:
                error = error + 1
        error_rate = error/batch_size
        error_arr.append(error_rate)
    
    print ("Error rate of each iteration: " + str(error_arr))
    print ("Average error rate:" + str(np.average(error_arr)))

In [114]:
def compute_pca(data):
    pca = PCA()
    pca_data = pca.fit_transform(data)
    return pca_data

In [128]:
data_rest, label_rest = load_data(-1)
data_rest, label_rest = binary_all_channel(data_rest, label_rest, 4)

data_4, label_4 = load_data(4)
data_4, label_4 = binary_all_channel(data_4, label_4, 4)

X = np.concatenate((data_rest, data_4))
y = np.concatenate((label_rest, label_4))

X_normalized = sklearn.preprocessing.normalize(X, norm='l2')
X_pca = compute_pca(X_normalized)

all_data, all_label = shuffle(X_pca, y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [129]:
cross_val(all_data, all_label, 100)

Error rate of each iteration: [0.01, 0.01, 0.03, 0.03, 0.01, 0.03, 0.03, 0.03, 0.03, 0.03, 0.04, 0.02]
Average error rate:0.024999999999999998


In [131]:
# Classify rest and a number n (0-9)
for i in range(10):
    data_rest, label_rest = load_data(-1)
    data_rest, label_rest = binary_all_channel(data_rest, label_rest, i)

    data_num, label_num = load_data(i)
    data_num, label_num = binary_all_channel(data_num, label_num, i)


    X = np.concatenate((data_rest, data_num))
    y = np.concatenate((label_rest, label_num))

    X_normalized = sklearn.preprocessing.normalize(X, norm='l2')
    X_pca = compute_pca(X_normalized)

    all_data, all_label = shuffle(X_pca, y)
    
    print("Digit" + str(i))
    cross_val(all_data, all_label, 100)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Digit0
Error rate of each iteration: [0.01, 0.06, 0.02, 0.01, 0.04, 0.06, 0.03, 0.04, 0.02, 0.01, 0.03, 0.04]
Average error rate:0.030833333333333338


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Digit1
Error rate of each iteration: [0.02, 0.04, 0.02, 0.01, 0.04, 0.0, 0.03, 0.04, 0.01, 0.03, 0.0, 0.02, 0.05]
Average error rate:0.023846153846153847


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

Digit7
Error rate of each iteration: [0.03, 0.03, 0.01, 0.01, 0.05, 0.11, 0.03, 0.04, 0.01, 0.02, 0.01, 0.05]
Average error rate:0.03333333333333333


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Digit8
Error rate of each iteration: [0.02, 0.03, 0.03, 0.03, 0.04, 0.01, 0.02, 0.03, 0.06, 0.02, 0.03, 0.02]
Average error rate:0.02833333333333334


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Digit9
Error rate of each iteration: [0.0, 0.02, 0.02, 0.04, 0.0, 0.02, 0.04, 0.02, 0.03, 0.02, 0.03, 0.01]
Average error rate:0.020833333333333332
